In [ ]:
import numpy as np
import pandas as pd
import h5py



import pybel
from tfbio.data import Featurizer

import warnings

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# path to the extracted PDBbind dataset
path = '/scratch/kanakala.ganesh/PDBBIND/'

In [ ]:
f = open('/scratch/kanakala.ganesh/PDBBIND/refined-set/index/INDEX_general_PL_data.2019')
lines = f.readlines()


In [ ]:
len(lines)

In [ ]:
f2 = open('affinity_data_custom.csv', 'w')
f2.write('pdbid -logKd/Ki' + '\n')
for i in range(len(lines)):
    l = lines[i].split()
    if (l[0] != "#"):
        f2.write(l[0] + ' '+ l[3] + '\n')
    else:
        print(lines[i])
f2.close()

In [ ]:
import os

In [ ]:
path1 = '/scratch/kanakala.ganesh/PDBBIND/v2019-other-PL/'
path2 = '/scratch/kanakala.ganesh/PDBBIND/refined-set/'

In [ ]:
general_list = os.listdir(path1)
refined_list = os.listdir(path2)

In [ ]:
len(general_list), len(refined_list)

In [ ]:
f3 = open('affinity_data_custom_clean.csv', 'w')
f3.write('pdbid -logKd/Ki class' + '\n')
c=0
for i in range(len(lines)):
    l = lines[i].split()
    if (l[0] != "#"):
        id = l[0]
        if id in general_list and id not in refined_list:
            f3.write(l[0] + ' '+ l[3] + ' ' +  'general'+'\n')
        elif id in refined_list and id not in general_list:
            f3.write(l[0] + ' '+ l[3] + ' ' + 'refined'+'\n')
        else:
            print(lines[i], "missing")
            print(c)
            c=c+1
    else:
        print(lines[i])
f3.close()

In [ ]:
path3= '/scratch/kanakala.ganesh/PDBBIND/CORE-FULL/ALL-CORE-affinities.csv'

In [ ]:
core_list_df = pd.read_csv(path3)

In [ ]:
core_list = core_list_df['id'].values

In [ ]:
affinity_data = pd.read_csv('affinity_data_custom_clean.csv', delimiter=' ')
# affinity_data = affinity_data[~np.in1d(affinity_data['pdbid'], list(missing))]
affinity_data.head()

In [ ]:
# Check for NaNs

affinity_data['-logKd/Ki'].isnull().any()

In [ ]:
for i in core_list:
    if i in general_list:
        general_list.remove(i)
    if i in refined_list:
        refined_list.remove(i)
        

In [ ]:
# # Separate core, refined, and general sets

# core_set = ! grep -v '#' $path/PDBbind_2016_plain_text_index/index/INDEX_core_data.2016 | cut -f 1 -d ' '
# core_set = set(core_set)

# refined_set = ! grep -v '#' $path/PDBbind_2016_plain_text_index/index/INDEX_refined_data.2016 | cut -f 1 -d ' '
# refined_set = set(refined_set)

# general_set = set(affinity_data['pdbid'])


# assert core_set & refined_set == core_set
# assert refined_set & general_set == refined_set

# len(general_set), len(refined_set), len(core_set)

In [ ]:
affinity_data['include'] = True

In [ ]:
# # Exclude v 2013 core set - it will be used as another test set

# core2013 = ! cat core_pdbbind2013.ids
# core2013 = set(core2013)

# affinity_data['include'] = True
# affinity_data.loc[np.in1d(affinity_data['pdbid'], list(core2013 & (general_set - core_set))), 'include'] = False

In [ ]:
affinity_data['set'] = affinity_data['class']

In [ ]:
affinity_data = affinity_data.drop(columns=['class'])

In [ ]:
affinity_data.loc[np.in1d(affinity_data['pdbid'], list(general_list)), 'set'] = 'general'

affinity_data.loc[np.in1d(affinity_data['pdbid'], list(refined_list)), 'set'] = 'refined'

affinity_data.loc[np.in1d(affinity_data['pdbid'], list(core_list)), 'set'] = 'core'

affinity_data.head()

In [ ]:
affinity_data[affinity_data['include']].groupby('set').apply(len).loc[['general', 'refined', 'core']]

In [ ]:
# Check affinity distributions

grid = sns.FacetGrid(affinity_data[affinity_data['include']], row='set', row_order=['general', 'refined', 'core'],
                     size=3, aspect=3)
grid.map(sns.distplot, '-logKd/Ki');

In [ ]:
affinity_data[['pdbid']].to_csv('pdb_custom.ids', header=False, index=False)

In [ ]:
affinity_data[['pdbid', '-logKd/Ki', 'set']].to_csv('affinity_data_cleaned_custom.csv', index=False)

# Parse molecules

In [ ]:
dataset_path = {'general': 'general-set-except-refined', 'refined': 'refined-set', 'core': 'refined-set'}

In [ ]:
path = '/'

In [ ]:
# %%bash 

# # Prepare pockets with UCSF Chimera - pybel sometimes fails to calculate the charges.
# # Even if Chimera fails to calculate several charges (mostly for non-standard residues),
# # it returns charges for other residues.

# for pdbfile in /scratch/kanakala.ganesh/PDBBIND/v2019-other-PL/*/*_pocket.pdb; do
#     mol2file=${pdbfile%pdb}mol2
# #     echo $m
# # ol2file 
# #     if [[ ! -f $mol2file ]]; then
#         echo $mol2file
#         echo -e "open $pdbfile \n addh \n addcharge \n write format mol2 0 tmp.mol2 \n stop" | bash /home2/jai/.local/UCSF-Chimera64-1.15rc/bin/chimera --nogui
#         # Do not use TIP3P atom types, pybel cannot read them
#         sed 's/H\.t3p/H    /' tmp.mol2 | sed 's/O\.t3p/O\.3  /' > $mol2file
# #     fi
# done 


In [ ]:
## %%bash -s $path

# # Prepare pockets with UCSF Chimera - pybel sometimes fails to calculate the charges.
# # Even if Chimera fails to calculate several charges (mostly for non-standard residues),
# # it returns charges for other residues.

# path=$1

# for dataset in general-set-except-refined refined-set; do
#     echo $dataset
#     for pdbfile in /scratch/kanakala.ganesh/2019/coreset/*/*_pocket.pdb; do
#         mol2file=${pdbfile%pdb}mol2
#         if [[ ! -e $mol2file ]]; then
#             echo -e "open $pdbfile \n addh \n addcharge \n write format mol2 0 tmp.mol2 \n stop" | chimera --nogui
#             # Do not use TIP3P atom types, pybel cannot read them
#             sed 's/H\.t3p/H    /' tmp.mol2 | sed 's/O\.t3p/O\.3  /' > $mol2file
#         fi
#     done 
# done > chimera_rw.log

In [ ]:
featurizer = Featurizer()
charge_idx = featurizer.FEATURE_NAMES.index('partialcharge')

path = '/scratch/kanakala.ganesh/PDBBIND/v2019-other-PL/'
ds_path = ''
# ds_path ="refined-set"
# ds_path= "v2019-other-PL"
data = affinity_data[affinity_data['set']=='general']    
dataset_name='general'
i=0
pocket_failed=[]
ligand_failed=[]
count = 0
with h5py.File('%s/%s.hdf' % (path, dataset_name), 'w') as f:
    for _, row in data.iterrows():
        
        if row['pdbid'] in general_list:
            
            name = row['pdbid']
            affinity = row['-logKd/Ki']
            include = row['include']
            ligand = next(pybel.readfile('mol2', '%s/%s/%s/%s_ligand.mol2' % (path, ds_path, name, name)))
            # do not add the hydrogens! they are in the strucutre and it would reset the charges

            try:
                pocket = next(pybel.readfile('mol2', '%s/%s/%s/%s_pocket.mol2' % (path, ds_path, name, name)))
                # do not add the hydrogens! they were already added in chimera and it would reset the charges
            except:
                warnings.warn('no pocket for %s (%s set)' % (name, dataset_name))
                print('count',count)
                count +=1 
                continue

            ligand_coords, ligand_features = featurizer.get_features(ligand, molcode=1)
            if not (ligand_features[:, charge_idx] != 0).any():
                ligand_failed.append(i)

            pocket_coords, pocket_features = featurizer.get_features(pocket, molcode=-1)
            if not (pocket_features[:, charge_idx] != 0).any():
                pocket_failed.append(i)

            centroid = ligand_coords.mean(axis=0)
            ligand_coords -= centroid
            pocket_coords -= centroid

            data = np.concatenate((np.concatenate((ligand_coords, pocket_coords)),
                                   np.concatenate((ligand_features, pocket_features))), axis=1)
    #         print(include)

            if row['include']:
                dataset = f.create_dataset(name, data=data, shape=data.shape, dtype='float32', compression='lzf')
                dataset.attrs['affinity'] = affinity
                i += 1
                print(i)
#             break
print('prepared', i, 'complexes')


In [ ]:
f.close()

In [ ]:
len(ligand_failed), len(pocket_failed)

In [ ]:
# featurizer = Featurizer()
# charge_idx = featurizer.FEATURE_NAMES.index('partialcharge')

# with h5py.File('%s/core2013.hdf' % path, 'w') as g:
#     j = 0

#     for dataset_name, data in affinity_data.groupby('set'):

#         print(dataset_name, 'set')
#         i = 0
#         ds_path = dataset_path[dataset_name]


#         with h5py.File('%s/%s.hdf' % (path, dataset_name), 'w') as f:
#             for _, row in data.iterrows():

#                 name = row['pdbid']
#                 affinity = row['-logKd/Ki']

#                 ligand = next(pybel.readfile('mol2', '%s/%s/%s/%s_ligand.mol2' % (path, ds_path, name, name)))
#                 # do not add the hydrogens! they are in the strucutre and it would reset the charges

#                 try:
#                     pocket = next(pybel.readfile('mol2', '%s/%s/%s/%s_pocket.mol2' % (path, ds_path, name, name)))
#                     # do not add the hydrogens! they were already added in chimera and it would reset the charges
#                 except:
#                     warnings.warn('no pocket for %s (%s set)' % (name, dataset_name))
#                     continue

#                 ligand_coords, ligand_features = featurizer.get_features(ligand, molcode=1)
#                 assert (ligand_features[:, charge_idx] != 0).any()
#                 pocket_coords, pocket_features = featurizer.get_features(pocket, molcode=-1)
#                 assert (pocket_features[:, charge_idx] != 0).any() 

#                 centroid = ligand_coords.mean(axis=0)
#                 ligand_coords -= centroid
#                 pocket_coords -= centroid

#                 data = np.concatenate((np.concatenate((ligand_coords, pocket_coords)),
#                                        np.concatenate((ligand_features, pocket_features))), axis=1)

#                 if row['include']:
#                     dataset = f.create_dataset(name, data=data, shape=data.shape, dtype='float32', compression='lzf')
#                     dataset.attrs['affinity'] = affinity
#                     i += 1
#                 else:
#                     dataset = g.create_dataset(name, data=data, shape=data.shape, dtype='float32', compression='lzf')
#                     dataset.attrs['affinity'] = affinity
#                     j += 1

#         print('prepared', i, 'complexes')
#     print('excluded', j, 'complexes')

In [ ]:
from sklearn.utils import shuffle


In [ ]:
with h5py.File('./DATA/general.hdf', 'r') as g, \
     h5py.File('./DATA/refined.hdf', 'r') as r, \
     h5py.File('./DATA/full.hdf', 'w') as f:
    for name in g:
        dataset= f.create_dataset(name, data=g[name])
        dataset.attrs['affinity'] = g[name].attrs['affinity']
    for name in r:
        dataset= f.create_dataset(name, data=r[name])
        dataset.attrs['affinity'] = r[name].attrs['affinity']

f.close()
g.close()
r.close()

In [ ]:
fold0 = pd.read_csv('../FOLDS/full_pk_cv_test0.types', sep=' ')
fold1 = pd.read_csv('../FOLDS/full_pk_cv_test1.types', sep=' ')
fold2 = pd.read_csv('../FOLDS/full_pk_cv_test2.types', sep=' ')

In [ ]:
fold_ids = list(fold0['pdbid'].values) + list(fold1['pdbid'].values) + list(fold2['pdbid'].values)

In [ ]:
import pickle

In [ ]:
with open('../folds_using_0.30_thresh_pocketmatch.pkl', 'rb') as f:
    folds = pickle.load(f)
    
# for fold in folds:
#     for pdbid in fold:
#         if pdbid not in fold_ids:
#             fold.remove(pdbid)

In [ ]:
[print(len(i)) for i in folds]

In [ ]:
with h5py.File('./DATA/full.hdf', 'r') as f, \
     h5py.File('./DATA/training_set.hdf', 'w') as t1, \
     h5py.File('./DATA/test_set.hdf', 'w') as t2  :
     for name in f:
        if name in folds[0] or name in folds[1]:
            data = t1.create_dataset(name, data=f[name])
            data.attrs['affinity'] = f[name].attrs['affinity']
        else:
            data = t2.create_dataset(name, data=f[name])
            data.attrs['affinity'] = f[name].attrs['affinity']
        
t1.close()
t2.close()            